In [70]:
import numpy as np
from blueqat import opt
%matplotlib inline


====================================  
大学生活における「最小努力で必要単位数を取得する」ケースについて考える。  
  
或る履修可能科目群があるとして、その履修によって得られる単位を$u_i$、  
それらを取得するために必要な工数（いわゆる「逆評定」の評価を数値化したもの）を$w_i$とし、  
科目$i$を履修するか否かを$x_i$で表すとする。  
  
最低でも$U$単位は取得しなければならないという前提条件にたつと、  
これは取得単位の制約条件を満たしつつ総工数を最小化する最適化問題と考えられる。  

====================================  


最小化対象:  
$ \displaystyle　\sum_{ i = 1 }^{ N } w_i x_i,　x_i \in \{0, 1\} $
<br>
<br>
subject to:  
$ \displaystyle　\sum_{ i = 1 }^{ N } u_i x_i \geq U $
<br>
<br>
$\implies$最小化対象:  
$ \displaystyle　\sum_{ i = 1 }^{ N } w_i x_i + \alpha (\sum_{ i = 1 }^{ N } u_i x_i - U)^2 $
<br>  
このままでは  
- 取得単位に1満たなかった場合
- 取得単位を1超えた場合  
を同様の重みで評価してしまうため、若干の変形を加える
<br>
<br>

$\implies$最小化対象（変形後）:  
$ \displaystyle　\sum_{ i = 1 }^{ N } w_i x_i + \alpha (\sum_{ i = 1 }^{ N } u_i x_i - (U + \sum_{ j = 1 }^{ 3 } \lambda_j))^2 $  
$\implies$  
$ \displaystyle　\mathbf{x}^T \mathbf{w} + \alpha\{\mathbf{x}^T \mathbf{u} - (5 + \mathbf{\lambda})\}^2,　\lambda_j \in \{0, 1\}  $  
<br>
<br>
※「高々最低取得単位数よりも3多い程度に収まるだろう」という想定のもと、チューニングパラメータ$\lambda$を3個に制限  
　⇒第1項の最小化を考えると$x_i=0　(for　\forall i)$ となってしまうため、$\alpha$値のバランス調整が難しい…

そして最適化対象の式に対して、  
$ \mathbf{P} = \mathbf{x}^T C \mathbf{x} $  
の形になるよう式変形を行う
<br>
<br>
$$
    \mathbf{C} =
        \left(\begin{array}{ccc}
            w_{1} + \alpha u_{1}^2 - 10u_{1}& 2\alpha u_{1}u_{2} & 2\alpha u_{1}u_{3} & \cdots & 2\alpha u_{1}u_{N} & -2\alpha u_{1} & \cdots　& & -2\alpha u_{1} \\
            & \ddots & 2\alpha u_{2}u_{3} & \ddots & \vdots & \vdots & & & \vdots \\
            & & \ddots & & 2\alpha u_{N-2}u_{N-1} \\
            & & & \ddots & 2\alpha u_{N-1}u_{N}\\
            & & & &  w_{N}+\alpha u_{N}^2 - 10u_{N} & -2\alpha u_{N} & \cdots　& & -2\alpha u_{N}  \\
            & & & & & 11\alpha & 2\alpha & \cdots & 2\alpha \\
            & & & & & & 11\alpha & & \vdots \\
            & & & & & & & \ddots & 2\alpha\\
            & & & & & & & & 11\alpha   \\
        \end{array}\right)
$$

In [71]:
# u_i : 科目iの履修による取得単位数
u = [1, 3, 2, 2, 1]

# w_i : 科目iの単位取得難易度＝単位取得所要時間（工数）
w = [1, 5, 3, 1, 2]

# 行列サイズ
N = len(u)
# 必要取得単位数 : 5で固定
# 最低取得単位数+3まで許容する
U = 3
# 最適化パラメータα
alpha = 1.5

下記のようなテーブルが与えられていると考える。  

この程度なら暗算でも確認可能であり、科目[A, C, D]の組み合わせが最小工数を取りそうである。  
つまりシミュレーション実行結果としては$[1,0,1,1,0]$が出てくれるのが望ましい。
<br>

|  科目  |  取得単位[単位]  |  想定工数[人月]  |
| ---- | ---- | ---- |
|  A  |  1.0  |  1.0  |
|  B  |  3.0  |  5.0  |
|  C  |  2.0  |  3.0  |
|  D  |  2.0  |  1.0  |
|  E  |  1.0  |  2.0  |
  
  
状況が整理できたので、QUBO形式である$C$を計算するため関数を定義する。

In [72]:
def get_qubo(u: list,w: list, N: int, U: int):
    
    # 行列の合計サイズ
    size = N + U
    qubo = np.zeros((size, size))
    
    # 1. x vector part
    # 対角成分
    for i in range(N):
        for j in range(N):
            if j == i:
                qubo[i][j] = w[i] + alpha * (u[i] ** 2) - 10 * alpha * u[i]
   # 上三角行列部分
    for i in range(N):
        for j in range(N):
            if j >= i + 1:
                qubo[i][j] = 2 * alpha * u[i] * u[j]
    for i in range(N):
        for j in range(N, N + U):
            qubo[i][j] = -2 * alpha * u[i]
                
    # 2. lambda vector part
    # 対角成分
    for i in range(N, N + U):
        for j in range(N, N + U):
            if j == i:
                qubo[i][j] = 11 * alpha
    # 上三角行列部分
    for i in range(N, N + U):
        for j in range(N, N + U):
            if j >= i + 1:
                qubo[i][j] = 2 * alpha
                
    return qubo

In [73]:
qubo = get_qubo(u, w, N, U)
qubo

array([[-12.5,   9. ,   6. ,   6. ,   3. ,  -3. ,  -3. ,  -3. ],
       [  0. , -26.5,  18. ,  18. ,   9. ,  -9. ,  -9. ,  -9. ],
       [  0. ,   0. , -21. ,  12. ,   6. ,  -6. ,  -6. ,  -6. ],
       [  0. ,   0. ,   0. , -23. ,   6. ,  -6. ,  -6. ,  -6. ],
       [  0. ,   0. ,   0. ,   0. , -11.5,  -3. ,  -3. ,  -3. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  16.5,   3. ,   3. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,  16.5,   3. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,   0. ,   0. ,  16.5]])

In [76]:
a = opt.opt() 
a.qubo = qubo
result = a.sa(shots = 100, sampler = "fast")
opt.counter(result)

Counter({'01010000': 29,
         '11100010': 1,
         '10110000': 55,
         '10011000': 10,
         '11010001': 1,
         '11010010': 1,
         '01100000': 3})

$\alpha$ = 1.0の場合  
Counter({'10110000': 25,
         '00110000': 31,
         '10011000': 24,
         '01010000': 19,
         '11000000': 1})  

$\alpha$ = 1.5の場合  
Counter({'01010000': 29,
         '11100010': 1,
         '10110000': 55,
         '10011000': 10,
         '11010001': 1,
         '11010010': 1,
         '01100000': 3})

$\alpha$ = 2.0の場合  
Counter({'01010000': 23,
         '10110000': 47,
         '10011000': 6,
         '01110110': 2,
         '11100100': 2,
         '11010100': 6,
         '11010010': 6,
         '01100000': 3,
         '11010001': 2,
         '11000000': 2,
         '01110101': 1})  

どうやら$\alpha$=1.5 辺りで$[1,0,1,1,0]$ の確率が最大となりそうである。  

$\alpha$ = 1.0の場合の次点である$[0,0,1,1,0],[1,0,0,1,1]$は単位数自体が足りていないため、ペナルティ項がうまく影響しきれていない。  
$\alpha$ = 2.0の場合の次点である$[0,1,0,1,0]$は単位数は満たしているものの、最適解より工数が1多いので局所最適解である。 